## Chapter 7: Hypothesis and Inference

## Testing if a coin is fair

In [27]:
from typing import Tuple
import math

In [28]:
def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Returns mu and sigma corresponding to Binomial(n,p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

##### Finding Probability Interval from CDF

In [29]:
from scipy.stats import norm

In [30]:
"""Normal CDF is probability that variable is below a given threshold"""
normal_probability_below = norm.cdf

In [50]:
def normal_probability_above(lo: float,
                            mu: float = 0,
                            sigma: float = 1) -> float:
    """The probability that N(mu,sigma) is greater than lo"""
    return 1 - norm.cdf(lo, mu, sigma)

In [32]:
def normal_probability_between(lo: float,
                              hi: float, 
                              mu: float = 0,
                              sigma: float = 1) -> float:
    """The probability that N(mu, sigma) is between lo and hi"""
    return norm.cdf(hi, mu, sigma) - norm.cdf(lo, mu, sigma)

In [33]:
def normal_probability_outside(lo: float,
                              hi: float,
                              mu: float = 0,
                              sigma: float = 1) -> float:
    """The probability that N(mu, sigma) is outside the range from lo to hi"""
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [34]:
cdf_inverse = norm.ppf

In [35]:
def normal_upper_bound(probability: float,
                      mu: float = 0,
                      sigma: float = 1) -> float:
    """Returns the z for which P(Z <= z) = probability"""
    return norm.ppf(probability, mu, sigma)

In [36]:
def normal_lower_bound(probability: float,
                      mu: float = 0,
                      sigma: float = 1) -> float:
    """Returns the z for which P(Z >= z) = probability"""
    return norm.ppf(1 - probability, mu, sigma)

In [37]:
def normal_two_sided_bounds(probability: float,
                           mu: float = 0,
                           sigma: float = 1) -> Tuple[float, float]:
    """Returns the symmetric bounds about the mean the contain the desired probability"""
    tail_probability = (1 - probability) / 2
    
    """Upper bound should have tail probability above it"""
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    """Lower bound should have tail probability below it"""
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return (lower_bound, upper_bound)

##### Testing the Coin

In [38]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [39]:
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [40]:
print("19/20 tests will result in a total between %s and %s." % (lower_bound, upper_bound))

19/20 tests will result in a total between 469.0102483847719 and 530.9897516152281.


But how significant of a change do we need?

In [41]:
"""Bounds assuming p is 0.5"""
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [42]:
"""Mu and sigma based on p = 0.55"""
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

In [43]:
"""A type 2 error occurs in the overlap between the new and H0 distribution"""
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability

print("The type 2 error probability is %s, so the power is %s" % (type_2_probability, power))

The type 2 error probability is 0.11345221890161725, so the power is 0.8865477810983827


How does this compare to a one-tailed test?

In [44]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability

"So, in the case of a one-tailed test, the type 2 probability is %s, and the power is %s." % (type_2_probability, power)

'So, in the case of a one-tailed test, the type 2 probability is 0.06362100214225695, and the power is 0.936378997857743.'

##### Using p-Values

In [45]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    """How likely likely are we to see a value at least as extreme as x?"""
    if x >= mu:
        return 2 * normal_probability_above(x, mu, sigma)
    if x <= mu:
        return 2 * normal_probability_below(x, mu, sigma)

In [53]:
"""Is 530 heads out of 1000 abnormal?"""
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

In [68]:
"""Let's run a simulation"""
import random
extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(1000))
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1

print("%s out of 1000 runs had extreme values" % extreme_value_count)

69 out of 1000 runs had extreme values


In [69]:
"""While 530 heads in this case is not considered extreme, let's check the p-Value of 532"""
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

The probability is less than 0.05, so we are able to reject H_0

In [72]:
"""To draw parallels between p-Values and previous methods"""
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [73]:
upper_p_value(524.5, mu_0, sigma_0) #Thus unable to reject the null

0.06062885772582072

In [75]:
upper_p_value(526.5, mu_0, sigma_0) #Thus able to reject the null

0.04686839508859242

##### Confidence Intervals

In [76]:
"""We need mu = p and sigma for confidence intervals. Without the true p, we approximate it from the existing run where
525/1000 flips came up heads"""
p_hat = 525/1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

In [77]:
"We are 95% confident that the following interval contains the true p"
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490098153452, 0.5559509901846548)

In [80]:
"""If the run had 540/1000 flips come up heads"""
p_hat = 540/1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
normal_two_sided_bounds(0.95, mu, sigma)

(0.5091095747672452, 0.5708904252327549)

Here, a fair coin is not within the confidence interval. 
In other words, in 95% of trials, H_0 is rejected.





##### p-Hacking

In [81]:
from typing import List

In [92]:
def run_experiment() -> List[bool]:
    """Flips a fair coin 1000 time, True = Heads, False = Tails"""
    return[random.random() < 0.5 for _ in range(1000)]

In [93]:
def reject_fairness(experiment: List[bool]) -> bool:
    """Using 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

In [94]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

print("The experiment rejected the null %s / 1000 times" % num_rejections)

The experiment rejected the null 46 / 1000 times


##### Running an A/B Test

In [95]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

Warning, comparing parameters of distributions usually requires sure knowledge of the sigma, not an estimate

In [98]:
def a_b_test_statistics(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [99]:
"""Example run"""
z = a_b_test_statistics(1000, 200, 1000, 180)

In [100]:
two_sided_p_value(z)

0.254141976542236

There is a 25% chance of observing this difference if there is no significant difference in distributions,
so we fail to reject the null.

In [101]:
z = a_b_test_statistics(1000, 200, 1000, 150)
two_sided_p_value(z)

0.0031896997062168583

There is only a 3% chance of this happening by chance, so we reject the null.

##### Bayesian Inference

Used here to make probability judgements on parameters of distributions

In [102]:
def B(alpha: float, beta: float) -> float:
    """A normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

In [105]:
def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:                                          #No weights outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

Here, we take an initial stance on the  distribution (fair? biased?) and then update the starting belief as more evidence is collected. Eventually, if enough evidence is collected, the posterior distribution will be the same no matter the prior distribution.

Note:Alpha and beta are essentially parameters for bias existing in the prior distribution.

Note: Binomial distribution is the conjugate prior to beta distribution. Thus, these distributions are appriopriate in the context of flipping a coin